In [1]:
import torch
from hydra import initialize, compose
from hydra.utils import instantiate
from bliss.surveys.dc2 import DC2
import tqdm

In [2]:
with initialize(config_path=".", version_base=None):
    notebook_cfg = compose("notebook_config")
dc2: DC2 = instantiate(notebook_cfg.surveys.dc2)
dc2.prepare_data()
dc2.setup()

dc2_train_dataloader = dc2.train_dataloader()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
n_sources_count = torch.zeros(10, device=device)
for batch in tqdm.tqdm(dc2_train_dataloader):
    n_sources_count += batch["tile_catalog"]["n_sources"].to(device=device).flatten().bincount(minlength=10)

100%|██████████| 3063/3063 [01:47<00:00, 28.56it/s]


In [4]:
print(n_sources_count)

tensor([7.7197e+07, 1.1920e+06, 1.0883e+04, 8.0000e+01, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00], device='cuda:0')
